In [194]:
import pandas as pd
import logging
import os
import sys
from dotenv import load_dotenv
from datetime import datetime
import psycopg2
import pyodbc


In [195]:
sys.path.insert(1, r'Q:\04 - PROJETOS\DW\Data-Warehouse\Func')
from utils2 import clear_env_variables, load_env, connect_to_database, execute_query_sql

In [196]:
def excel_file(file_path, sheet_name):
    """
    Função para ler uma planilha específica de um arquivo Excel e retornar um DataFrame.

    Parâmetros:
    file_path (str): Caminho do arquivo Excel.
    sheet_name (str): Nome da planilha a ser lida.

    Retorna:
    DataFrame: Dados da planilha como um DataFrame.
    """
    # Ler a sheet específica
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    return df

# Exemplo de uso da função
file_path = r'Q:\04 - PROJETOS\DW\Data-Warehouse\Documentos\Mapeamento das Fontes de Dados\CADASTRO\CADASTRO.xlsx'
sheet_name = 'ESTRUTURA_DW_CADASTRO OFICIAL'
df_cadastro = excel_file(file_path, sheet_name)
df_cadastro['DATA_ATUALIZAÇÃO'] = datetime.now()
df_cadastro['DATA_CADASTRO'] = datetime.now()

# Formatando as colunas para string com milissegundos
df_cadastro['DATA_ATUALIZAÇÃO'] = pd.to_datetime(df_cadastro['DATA_ATUALIZAÇÃO']).dt.strftime('%Y-%m-%d %H:%M:%S').str[:19]
df_cadastro['DATA_CADASTRO'] = pd.to_datetime(df_cadastro['DATA_CADASTRO']).dt.strftime('%Y-%m-%d %H:%M:%S').str[:19]

# Reconversão das colunas de volta para datetime64[ns]
df_cadastro['DATA_ATUALIZAÇÃO'] = pd.to_datetime(df_cadastro['DATA_ATUALIZAÇÃO'], format='%Y-%m-%d %H:%M:%S') #.dt.date
df_cadastro['DATA_CADASTRO'] = pd.to_datetime(df_cadastro['DATA_CADASTRO'], format='%Y-%m-%d %H:%M:%S') #.dt.date


In [197]:
# Definir tipos de dados de acordo com a tabela CADASTRO
column_types = {
    'ID_CADASTRO': int,
    'EMPRESA': str,
    'CLIENTE': str,
    'COD_CLIENTE': int,
    'COD_CARTEIRA': int,
    'COD_OP': int,
    'COD_CRM': int,
    'CARTEIRA': str,
    'OPERAÇÃO': str,
    'CRM': str,
    'SERVIDOR_CRM': str,
    'BANCO_CRM': str,
    'TIPO_CRM': str,
    'STATUS': str
}

def convert_and_fill(df, column_types):
    for column, dtype in column_types.items():
        if column not in df.columns:
            df[column] = pd.NA
        df[column] = df[column].astype(dtype, errors='ignore')
    return df

# Converter e preencher os tipos de dados no DataFrame
df_cadastro = convert_and_fill(df_cadastro, column_types)

display(df_cadastro.head())

,ID_CADASTRO,EMPRESA,CLIENTE,COD_CLIENTE,COD_CARTEIRA,COD_OP,COD_CRM,CARTEIRA,OPERAÇÃO,CRM,SERVIDOR_CRM,BANCO_CRM,TIPO_CRM,STATUS,DATA_CADASTRO,DATA_ATUALIZAÇÃO
0,1,MF,ENEL,1,1,1,200,ENEL CE,ENEL COB NAO ENERGIA,ACTYON,10.10.222.100,dbActyon_Enel,SQL SERVER,ATIVO,2024-07-31 13:40:48,2024-07-31 13:40:48
1,2,MF,ENEL,1,1,2,203,ENEL CE,ENEL CE VENCIDOS,ACTYON,10.10.222.100,dbActyon_Enel,SQL SERVER,ATIVO,2024-07-31 13:40:48,2024-07-31 13:40:48
2,3,MF,ENEL,1,1,3,189,ENEL CE,ENEL CE UNPAID,ACTYON,10.10.222.100,dbActyon_Enel,SQL SERVER,ATIVO,2024-07-31 13:40:48,2024-07-31 13:40:48
3,4,MF,ENEL,1,1,4,176,ENEL CE,ENEL CE PC6,ACTYON,10.10.222.100,dbActyon_Enel,SQL SERVER,ATIVO,2024-07-31 13:40:48,2024-07-31 13:40:48
4,5,MF,ENEL,1,1,5,175,ENEL CE,ENEL CE PC5,ACTYON,10.10.222.100,dbActyon_Enel,SQL SERVER,ATIVO,2024-07-31 13:40:48,2024-07-31 13:40:48


In [198]:
df_cadastro.dtypes

ID_CADASTRO                  int32
EMPRESA                     object
CLIENTE                     object
COD_CLIENTE                  int32
COD_CARTEIRA                 int32
COD_OP                       int32
COD_CRM                      int32
CARTEIRA                    object
OPERAÇÃO                    object
CRM                         object
SERVIDOR_CRM                object
BANCO_CRM                   object
TIPO_CRM                    object
STATUS                      object
DATA_CADASTRO       datetime64[ns]
DATA_ATUALIZAÇÃO    datetime64[ns]
dtype: object

In [199]:
dotenv_path = r'Q:\04 - PROJETOS\DW\Data-Warehouse\Environment\.Env_18'
# Limpando as variáveis de ambiente anterior
clear_env_variables()
# Carregue as variáveis de ambiente do arquivo .env
load_dotenv(dotenv_path)

# Acesse as variáveis de ambiente
server = os.getenv('SERVER_MIS')
database = os.getenv('DATABASE_MIS')
#username = os.getenv('USERNAME_MIS')
#password = os.getenv('PASSWORD_MIS')
driver = os.getenv('DRIVER_MIS')
port = os.getenv('PORT_MIS')

# Exemplo de uso das variáveis
print(f"Conectando ao servidor {server}...")

# Conectar ao banco de dados usando pyodbc
connection_string = (
    f"DRIVER={driver};"
    f"SERVER={server},{port};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes;"
)

connection = pyodbc.connect(connection_string)
print("Conexão bem-sucedida!")

def merge_cadastro():
    global connection, df_cadastro

    def merge_data_with_sql(connection, script_file, data):
        try:
            cursor = connection.cursor()
            with open(script_file, 'r', encoding='utf-8') as file:
                sql_script = file.read()

                for row in data:
                    cursor.execute(sql_script, row)
            connection.commit()
            print("Merge dos dados executado com sucesso.")
        except Exception as error:
            print(f"Erro ao executar o merge dos dados: {error}")

    try:
        if df_cadastro is not None:
            data = df_cadastro.values.tolist()  # Converte DataFrame para lista de listas

            if connection is not None:
                script_file_path = r'Q:\04 - PROJETOS\DW\Data-Warehouse\Source\CADASTRO\CADASTRO_MERGE.SQL'
                merge_data_with_sql(connection, script_file_path, data)
            else:
                print("Erro: Conexão com o banco de dados não foi estabelecida.")
        else:
            print("Erro: DataFrame não foi carregado.")

    except Exception as error:
        print(f"Erro ao conectar ou executar o merge dos dados: {error}")
    finally:
        connection.close()

merge_cadastro()

Conectando ao servidor SRVMIS06...
Conexão bem-sucedida!
Merge dos dados executado com sucesso.
